In [ ]:
import numpy as np
import base64
import io
import os
import requests
import json
from PIL import Image as Images,ImageFont,ImageDraw
from IPython.display import Image, clear_output, display
import cv2
%matplotlib inline
from matplotlib import pyplot as plt
import time
from os import listdir
from os.path import isfile, isdir, join

In [ ]:

#Setup image path and inference server
datasetPath = "./test/"
resultPath = "./result/"
inferenceIP = "myinference-i-clusterip-svc"
inferencePort = "9999"

In [ ]:
#Font setup
fontpath = "/yolov7/cht.otf"
color = (255, 0, 0)
font = ImageFont.truetype(fontpath, 20)

In [ ]:
def arrayShow(imageArray):
    resized = cv2.resize(imageArray, (500, 333), interpolation=cv2.INTER_CUBIC)
    ret, png = cv2.imencode('.png', resized)
    return Image(data=png)

In [ ]:
# 取得所有檔案與子目錄名稱
if not os.path.exists(resultPath):
    os.makedirs(resultPath)
files = listdir(datasetPath)

for f in files:
    fullpath = join(datasetPath, f)
    if isfile(fullpath):
        output = join(resultPath, f)
        with open(fullpath, "rb") as inputFile:
            data = inputFile.read()
        body = {"image": base64.b64encode(data).decode("utf-8"), "thresh": 0.5, "getImage": False}
        res = requests.post("http://" + inferenceIP +":" + inferencePort + "/yolov7/detect", json=body)
        detected = res.json()
        objects_json = json.loads(detected.get('results'))

        oriImage = cv2.imread(fullpath)
        img_pil = Images.fromarray(cv2.cvtColor(oriImage, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(img_pil)
        for obj in objects_json:
            xmin = (obj['points'][0])
            ymin = (obj['points'][1])
            xmax = (obj['points'][2])
            ymax = (obj['points'][3])
            label = (obj['label'])
            confidence = obj['confidence']
            pos = tuple((int(xmin),int(ymin)))
            box =  (int(xmin), int(ymax)), (int(xmax), int(ymin))
            draw.text(pos, label, font = font, fill = color)
            draw.rectangle(box,outline="green")
        cv_img = cv2.cvtColor(np.asarray(img_pil),cv2.COLOR_RGB2BGR)
        cv2.imwrite(output, cv_img) 
        img = arrayShow(cv_img)
        clear_output(wait=True)
        display(img)
        time.sleep(1)